In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#3.10.7

Here I loaded in two seperate data file's having to accout for some latin characters

In [ ]:
Rolling_stone = pd.read_csv("albumlist.csv", encoding='latin-1',)
Rate_My_Music= pd.read_csv("rym_top_5000_all_time.csv", encoding='latin-1',)
Rate_My_Music.head()


Here I loaded I joined my two data sets and cleaned some data by renaming some columns in Rate_My_Music to match some columns in Rolling_stone for and easier merge

In [ ]:
Rate_My_Music = Rate_My_Music.rename(columns={"Artist Name": "Artist", "Genres": "Genre"})
joined_data = pd.merge(Rolling_stone, Rate_My_Music, on=['Album', 'Artist'], how='inner')
joined_data = joined_data.drop('Genre_y', axis=1)
joined_data = joined_data.rename(columns={'Genre_x': 'Genre'})
joined_data.head(8)


This where I viuslized my Data I started by showing a Bar Graph of Top 10 Genres

In [ ]:
top_10_genres = joined_data['Genre'].value_counts().nlargest(10)
plt.figure(figsize=(10, 6))
top_10_genres.plot(kind='bar')
plt.title('Top 10 Genres')
plt.xlabel('Genre')
plt.ylabel('Albums')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(joined_data['Year'], joined_data['Average Rating'], alpha=0.5)
plt.title('Average Ratings Based on Release Year')
plt.xlabel('Release Year of Album')
plt.ylabel('Album Rating')
plt.xticks(rotation=45)
plt.show()

In [ ]:

top_10_artists = joined_data['Artist'].value_counts().nlargest(10)

plt.figure(figsize=(10, 6))
top_10_artists.plot(kind='bar')
plt.title('Top 10 Artists')
plt.xlabel('Artist')
plt.ylabel('Number of Albums')
plt.xticks(rotation=45)
plt.show()